# Training the model

### Imports

In [117]:
import glob
import numpy as np


from pprint import pprint
from collections import deque

from sklearn.model_selection import KFold

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Activation, concatenate
from keras.utils import np_utils, normalize, to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import *

import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

## Hyperparameters

In [118]:
SEQ_LEN = 3
FUTURE_SEQ_PREDICT = 1

In [119]:
def shuffle(arr):
    return np.random.shuffle(arr)

In [136]:
'''
train
    Trains the given model

    Uses EarlyStopping
inputs:
    model
    input data
    batch_size
    epochs
returns:
    the trained model
'''
def train(model, x_note_train, x_len_train, y_note_train, y_len_train, batch_size, epochs):

    tensorboard = TensorBoard(log_dir="./logs/tensorboard", histogram_freq=0, write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
    earlystopping = EarlyStopping(monitor='output_note_acc', min_delta=0, patience=1, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

    filepath = "../../model-checkpoint.h5"  # unique file name that will include the epoch and the validation acc for that epoch
    # checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

    checkpoint = ModelCheckpoint(
        filepath=filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    # Train model
    history = model.fit(
        {'input_notes': x_note_train, 'input_length': x_len_train},
        {'output_note': y_note_train, 'output_length': y_len_train},
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        # validation_data=(
        #     [x_test[:,:,0,:], x_test[:,:,1,:]],
        #     [y_test[:,0,0,:], y_test[:,0,1,:]]),
        callbacks=[tensorboard, checkpoint, earlystopping]
    )

    return model

'''
getModel
    Creates an LSTM model
retuns:
    keras model
'''
def getModel(seqLen):
    input_notes = Input(shape=(seqLen,87), name="input_notes")
    input_length = Input(shape=(seqLen,53), name="input_length")
    inputs = concatenate([input_notes, input_length])

    model = LSTM(128, return_sequences=True)(inputs)
    model = Dropout(0.2)(model)
    model = LSTM(128, return_sequences=False)(model)
    model = Dropout(0.2)(model)

    output_note = Dense(87, activation="softmax", name='output_note')(model)
    output_length = Dense(53, activation="softmax", name="output_length")(model)

    model = Model(inputs=[input_notes, input_length], outputs=[output_note, output_length])
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

    return model



'''
getData
inputs:
    filepath for the dataset
    network input sequence length
    network prediction sequence length
returns:
    x_note, x_len - notes and note duration in an array of length seq_len
    y_note, y_len - notes and note duration in an array of length prediction_seq_len
'''
def getData(input_filepath, seq_len, prediction_seq_len):
    # noOfInputData = 100000
    x_note = []
    x_len = []
    y_note = []
    y_len = []

    for file in glob.glob(f"{input_filepath}*.npy"):
        # if len(x) >= noOfInputData:
        #     break

        x_data, y_data = createXY(np.load(file), seq_len, prediction_seq_len)

        if x_data == [] or y_data == []:
            continue
        if x_data.ndim == 3:
#             print(np.array(x_data).shape)
            x_note += list(x_data[:,:,0])
            x_len += list(x_data[:,:,1])
            y_note += list(y_data[:,:,0])
            y_len += list(y_data[:,:,1])
    
    x_note = np.array(x_note).reshape(299643, 2, 1)
    x_len = np.array(x_len).reshape(299643, 2, 1)
    
    return x_note, x_len, y_note, y_len


'''
createXY
    gets sequences of data
inputs:
    np array of notes and duration
    network input sequence length
    network prediction sequence length
returns:
    x - notes and note duration in an array of length seq_len
    y - notes and note duration in an array of length prediction_seq_len
'''
def createXY(arr, sequenceLength, predictionLength):
    x = []
    y = []
    
    skipRests = True if arr[0][0] == 0 else False
    
    i = 0
    while i+sequenceLength+predictionLength < len(arr):
        
        if skipRests:
            if arr[i][0] == 0:
                i += sequenceLength
                continue
            else:
                skipRest = False
        
        x.append(arr[i : i+sequenceLength])
        y.append(arr[i+sequenceLength : i+sequenceLength+predictionLength])
        i += sequenceLength

    np.random.seed(1)
    np.random.shuffle(x)
    np.random.seed(1)
    np.random.shuffle(y)

    return (np.array(x), np.array(y))

In [139]:
input_filepath = "../../data/processed-130000/"
seq_len = 2
prediction_seq_len =  1
batch_size = 64
epochs = 10

x_note, x_len, y_note, y_len = getData(input_filepath, seq_len, prediction_seq_len)
print(np.array(x_note).shape)
np.save("s2p1_full/x_note", x_note)
np.save("s2p1_full/x_len", x_len)
np.save("s2p1_full/y_note", y_note)
np.save("s2p1_full/y_len", y_len)

# x_note = to_categorical(x_note, 87)
# x_len = to_categorical(x_len, 53)
# y_note = to_categorical(y_note, 87)
# y_len = to_categorical(y_len, 53)


# # 80/20 split for training/testing dataset
# trainLength = int(round((len(x_note) * 4)/5))
# x_note_train = x_note[:trainLength]
# x_len_train = x_len[:trainLength]
# x_note_test = x_note[trainLength:]
# x_len_test = x_len[trainLength:]

# y_note_train = y_note[:trainLength]
# y_len_train = y_len[:trainLength]
# y_note_test = y_note[trainLength:]
# y_len_test = y_len[trainLength:]

# # NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
# print('train samples', x_note_train.shape)
# print('test samples', x_note_test.shape)

# model = getModel(seq_len)
# model.summary()

# model = train(model, x_note_train, x_len_train, y_note_train, y_len_train, batch_size, epochs)

# # plot_model(model, to_file='model.png')

# # Score model
# score = model.evaluate([x_note_test, x_len_test], [y_note_test, y_len_test], verbose=1)
# i = 0
# while i < len(model.metrics_names):
#     print(f"{model.metrics_names[i]}:",score[i])
#     i+=1
# # Save model
# model_name = "../../models/model-test5.hdf5"
# model.save(model_name)


print("Done")

/home/fireredninja/anaconda3/envs/generateMusicProject/lib/python3.6/site-packages/ipykernel_launcher.py:94: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/home/fireredninja/anaconda3/envs/generateMusicProject/lib/python3.6/site-packages/ipykernel_launcher.py:94: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(299643, 2, 1)
Done


In [ ]:
path = "./numpyRepresentation/"
noOfInputData = 100000
x = []
y = []

for file in glob.glob(f"{path}*.npy"):
    if len(x) >= noOfInputData:
        break
    
    x_data, y_data = createXY(np.load(file), 3, 1)
#     print(x_data.ndim)
    if x_data.ndim == 3:
        x += list(x_data)
        y += list(y_data)
    
x = to_categorical(x)
y = to_categorical(y)

In [ ]:
# 80/20 split for training/testing dataset
trainLength = int(round((len(x) * 4)/5))
x_train = x[:trainLength]
x_test = x[trainLength:]
y_train = y[:trainLength]
y_test = y[trainLength:]

In [ ]:
pprint(x_train[:,:,1,:].shape)

In [ ]:
# NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

from keras.utils import plot_model
from time import time


input_notes = Input(shape=(3,120), name="input_notes")
input_length = Input(shape=(3,120), name="input_length")
inputs = concatenate([input_notes, input_length])

model = LSTM(128, return_sequences=True)(inputs)
model = Dropout(0.2)(model)
model = LSTM(128, return_sequences=False)(model)
model = Dropout(0.2)(model)

output_note = Dense(114, activation="softmax", name='output_note')(model)
output_length = Dense(114, activation="softmax", name="output_length")(model)

model = Model(inputs=[input_notes, input_length], outputs=[output_note, output_length])
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


# model = Sequential()
# model.add(LSTM(256, input_shape=x_train.shape[1:], return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256, return_sequences=False))
# model.add(Dropout(0.2))
# # model.add(Dense(32))
# # model.add(Reshape((1, 128)))
# model.add(Dense(2, activation='softmax'))
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy']
# )
# model.summary()

# plot_model(model, to_file='model.png')

tensorboard = TensorBoard(log_dir="./logs/{}".format(time()), histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

filepath = "RNN_Final-{epoch:02d}"  # unique file name that will include the epoch and the validation acc for that epoch
# checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min',
)

# Train model
history = model.fit(
    {'input_notes': x_train[:,:,0,:], 'input_length': x_train[:,:,1,:]},
    {'output_note': y_train[:,0,0,:], 'output_length': y_train[:,0,1,:]},
    batch_size=32,
    epochs=100,
    validation_data=(
        [x_test[:,:,0,:], x_test[:,:,1,:]],
        [y_test[:,0,0,:], y_test[:,0,1,:]]),
    callbacks=[tensorboard, checkpoint, earlystopping]
)

In [ ]:
# Score model
score = model.evaluate([x_test[:,:,0,:], x_test[:,:,1,:]], [y_test[:,0,0,:], y_test[:,0,1,:]], verbose=1)
i = 0
while i < len(model.metrics_names):
    print(f"{model.metrics_names[i]}:",score[i])
    i+=1
# Save model
model.save("models/model")

## Plot training and validation scores

In [ ]:
# plotting
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['output_note_acc'])
plt.plot(history.history['val_output_note_acc'])
plt.plot(history.history['output_length_acc'])
plt.plot(history.history['val_output_length_loss'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Note Train', 'Note Test', 'Length Train', 'Length Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['output_note_loss'])
plt.plot(history.history['val_output_note_loss'])
plt.plot(history.history['output_length_loss'])
plt.plot(history.history['val_output_length_loss'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Note Train', 'Note Test', 'Length Train', 'Length Test', "Val", "Val Loss"], loc='upper left')
plt.show()

In [ ]:
model = model.load("../../models/model-2")
data = np.load("./numpyRepresentation/Digimon - Butterfly (Piano Version)0.npy")
data = to_categorical(data)
data_notes = np.reshape(data[:,0,:], (63, 3 , 103))
data_length = np.reshape(data[:,1,:], (63, 3, 103))

In [ ]:
prediction = model.predict([data_notes, data_length])
print(prediction)
# np.save("./prediction", np.reshape(prediction, (63,2)))

In [ ]:
# pprint(prediction.reshape(63,2))
def generateNotes(model, sequence):
    return model.predict(sequence)

In [ ]:
    PREDICTION_LENGTH = 20

    prediction = np.reshape(prediction, (63,2))

    i = 0
    input_sequence = np.array([prediction[:SEQ_LEN]])
    output_sequence = input_sequence.copy()
    # print(input_sequence)
    while i < PREDICTION_LENGTH:
        generated_notes = generateNotes(model, input_sequence)
        output_sequence = np.append(output_sequence, generated_notes, axis=1)
        input_sequence = np.append(input_sequence, generated_notes, axis=1)
        input_sequence = np.delete(input_sequence, 0, 1)
        i+=1

In [ ]:
from keras.utils import to_categorical

arr1 = np.array([[[1, 6], [2, 7 ],[3, 8]]])
arr2 = np.array([[[0.34858927,0.6514107 ]]])

# print("Shape: ", arr1.shape, arr2.shape)

pprint(to_categorical(arr1))
result = np.append(arr1,arr2, axis=1)
# pprint(result)
# np.delete(result, 0, 1)

In [ ]:
testArr = [[[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0], [0,0,0,0,0,1,0,0,0,0,0,0,0,]],
          [[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0], [0,0,0,0,0,1,0,0,0,0,0,0,0,]]]
np.array(testArr, dtype=np.float32)